In [ ]:
#Search for reads with both desired flanking sequence from input .fastq; counts and exports counts to a .csv output
#Also exports a report .txt file with information on how many reads match each flanking sequence for troubleshooting

import csv

def count_mismatches(sequence, target):
    """Count the number of mismatches between a sequence and a target sequence."""
    mismatches = sum(1 for a, b in zip(sequence, target) if a != b)
    return mismatches + abs(len(sequence) - len(target))

def find_sequences_in_reads(upstream_seq, downstream_seq, fastq_file):
    sequences = {}
    reads_with_flanking = 0
    reads_without_flanking = 0
    reads_with_only_upstream = 0
    reads_with_only_downstream = 0
    mismatches_upstream = {}
    mismatches_downstream = {}
    length_distribution = {}

    with open(fastq_file, 'r') as f:
        while True:
            # Read four lines at a time (ID, sequence, "+", quality)
            read_id = f.readline().strip()
            sequence = f.readline().strip()
            _ = f.readline()  # Skip the third line ("+")
            _ = f.readline()  # Skip the quality scores line
            
            # Check if we've reached the end of the file
            if not read_id:
                break
            
            # Find upstream and downstream sequences
            upstream_index = sequence.find(upstream_seq)
            downstream_index = sequence.find(downstream_seq)
            
            # If both sequences are found, extract the sequence between them
            if upstream_index != -1 and downstream_index != -1:
                reads_with_flanking += 1
                start_index = upstream_index + len(upstream_seq)
                end_index = downstream_index
                extracted_sequence = sequence[start_index:end_index]
                extracted_length = len(extracted_sequence)
                
                # Update the count for the extracted sequence
                if extracted_sequence in sequences:
                    sequences[extracted_sequence] += 1
                else:
                    sequences[extracted_sequence] = 1
                
                # Update the length distribution
                if extracted_length in length_distribution:
                    length_distribution[extracted_length] += 1
                else:
                    length_distribution[extracted_length] = 1
            else:
                reads_without_flanking += 1
                if upstream_index != -1:
                    reads_with_only_upstream += 1
                if downstream_index != -1:
                    reads_with_only_downstream += 1

                # Count mismatches for upstream and downstream sequences separately
                if upstream_seq not in sequence:
                    mismatches_up = count_mismatches(sequence[:len(upstream_seq)], upstream_seq)
                    mismatches_upstream[mismatches_up] = mismatches_upstream.get(mismatches_up, 0) + 1

                if downstream_seq not in sequence:
                    mismatches_down = count_mismatches(sequence[-len(downstream_seq):], downstream_seq)
                    mismatches_downstream[mismatches_down] = mismatches_downstream.get(mismatches_down, 0) + 1
    
    return (sequences, reads_with_flanking, reads_without_flanking, 
            reads_with_only_upstream, reads_with_only_downstream, 
            mismatches_upstream, mismatches_downstream, length_distribution)

def export_counts_to_csv(sequences_count, output_csv_file):
    with open(output_csv_file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["Sequence", "Count"])
        for sequence, count in sequences_count.items():
            csvwriter.writerow([sequence, count])

def write_report(reads_with_flanking, reads_without_flanking, reads_with_only_upstream, reads_with_only_downstream, mismatches_upstream, mismatches_downstream, length_distribution, output_report_file):
    with open(output_report_file, 'w') as reportfile:
        reportfile.write(f"Reads with flanking sequences: {reads_with_flanking}\n")
        reportfile.write(f"Reads without flanking sequences: {reads_without_flanking}\n")
        reportfile.write(f"Reads with only upstream sequence: {reads_with_only_upstream}\n")
        reportfile.write(f"Reads with only downstream sequence: {reads_with_only_downstream}\n")
        
        reportfile.write("Mismatches for upstream sequence:\n")
        for mismatches, count in sorted(mismatches_upstream.items()):
            reportfile.write(f"  {mismatches} mismatches: {count} reads\n")
        
        reportfile.write("Mismatches for downstream sequence:\n")
        for mismatches, count in sorted(mismatches_downstream.items()):
            reportfile.write(f"  {mismatches} mismatches: {count} reads\n")

        reportfile.write("Length distribution of sequences between flanking sequences:\n")
        for length, count in sorted(length_distribution.items()):
            reportfile.write(f"  Length {length}: {count} sequences\n")

# Example usage:
upstream_sequence = "GGTTGGCCAAGGATCCGACA"
downstream_sequence = "CATATCCAGTACACTTTGAG"
fastq_file = "E106.fastq"
output_csv_file = "E106.csv"
output_report_file = "E106_report.txt"

(sequences_count, reads_with_flanking, reads_without_flanking, 
 reads_with_only_upstream, reads_with_only_downstream, 
 mismatches_upstream, mismatches_downstream, length_distribution) = find_sequences_in_reads(upstream_sequence, downstream_sequence, fastq_file)

# Export the count for each extracted sequence to a CSV file
export_counts_to_csv(sequences_count, output_csv_file)

# Write the report to a text file
write_report(reads_with_flanking, reads_without_flanking, reads_with_only_upstream, reads_with_only_downstream, mismatches_upstream, mismatches_downstream, length_distribution, output_report_file)

# Print the count for each extracted sequence (for verification)
#for sequence, count in sequences_count.items():
#    print(f"Sequence: {sequence}, Count: {count}")

In [ ]:
#Filters position of interest sequences to a desired # nt and translate to amino acid sequence

import csv
from Bio.Seq import Seq

def translate_sequences(input_csv_file, nt_length, output_csv_file):
    translated_sequences = {}

    # Read the input CSV file
    with open(input_csv_file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        header = next(csvreader)  # Skip header

        for row in csvreader:
            sequence = row[0]
            count = int(row[1])

            # Filter sequences by the given nucleotide length
            if len(sequence) == nt_length:
                dna_seq = Seq(sequence)
                aa_seq = str(dna_seq.translate())

                # Update the count for the translated sequence
                if aa_seq in translated_sequences:
                    translated_sequences[aa_seq] += count
                else:
                    translated_sequences[aa_seq] = count

    # Write the translated sequences and counts to the output CSV file
    with open(output_csv_file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["Amino Acid Sequence", "Count"])
        for aa_seq, count in translated_sequences.items():
            csvwriter.writerow([aa_seq, count])

# Example usage
input_csv_file = "E106.csv"
nt_length = 9  # Example nucleotide length to filter by
output_csv_file = input_csv_file.replace(".csv", "_AA.csv")

translate_sequences(input_csv_file, nt_length, output_csv_file)

print(f"Translated sequences have been saved to {output_csv_file}")

In [ ]:
#Aligns data from two _AA.csv files to a single .csv file

import csv

def compare_aa_counts(file1, file2, output_file):
    # Read the first file into a dictionary
    counts1 = {}
    with open(file1, 'r') as csvfile1:
        csvreader = csv.reader(csvfile1)
        header1 = next(csvreader)  # Skip header
        for row in csvreader:
            aa_sequence = row[0]
            count = int(row[1])
            counts1[aa_sequence] = count

    # Read the second file into a dictionary
    counts2 = {}
    with open(file2, 'r') as csvfile2:
        csvreader = csv.reader(csvfile2)
        header2 = next(csvreader)  # Skip header
        for row in csvreader:
            aa_sequence = row[0]
            count = int(row[1])
            counts2[aa_sequence] = count

    # Combine the counts from both files
    combined_counts = {}
    for aa_sequence in set(counts1.keys()).union(set(counts2.keys())):
        combined_counts[aa_sequence] = [counts1.get(aa_sequence, 0), counts2.get(aa_sequence, 0)]

    # Write the combined counts to the output CSV file
    with open(output_file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["Amino Acid Sequence", "Count in File 1", "Count in File 2"])
        for aa_sequence, counts in combined_counts.items():
            csvwriter.writerow([aa_sequence, counts[0], counts[1]])

# Example usage
file1 = "E105_AA.csv"
file2 = "E106_AA.csv"
output_file = "E105_E106.csv"

compare_aa_counts(file1, file2, output_file)

print(f"Comparison of amino acid sequences has been saved to {output_file}")

In [ ]:
#Calculates raw fold-enrichment values and converts .csv to .xlxs

import os
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill

# Function to process a single CSV file and convert it to XLSX
def process_file(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Remove rows where 'Amino Acid Sequence' contains 'X'
    df = df[~df['Amino Acid Sequence'].str.contains('X')]

    # Calculate totals for 'Count in File 1' (column 2) and 'Count in File 2' (column 3)
    total_file1 = df['Count in File 1'].sum()
    total_file2 = df['Count in File 2'].sum()

    # Add the Fold-enrichment column
    df['Fold-enrichment'] = (df['Count in File 2'] / total_file2) / (df['Count in File 1'] / total_file1)

    # Create ExcelWriter object and write DataFrame to Excel
    output_file = file_path.replace('.csv', '.xlsx')
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Sheet1')

        # Access the workbook and the worksheet
        workbook = writer.book
        worksheet = writer.sheets['Sheet1']

        # Highlight rows where 'Count in File 1' < 100 (column B is 'Count in File 1')
        yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
        for row in range(2, len(df) + 2):  # Start from row 2 (Excel is 1-indexed)
            cell_value = worksheet.cell(row=row, column=2).value
            if cell_value < 100:
                for col in range(1, 5):  # Apply fill to columns A, B, C, and D
                    worksheet.cell(row=row, column=col).fill = yellow_fill

    print(f"Processed {file_path} -> {output_file}")

# Function to process all CSV files in a directory
def process_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            process_file(file_path)

# Example usage
directory = r"/path/to/directory"  # Replace with your directory path
process_directory(directory)

In [ ]:
#Used to sort single-mutant data for each position, can be used to sort out any given sequence,

import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill

def sort_amino_acid_sequences(input_file, output_file, position_criteria):
    # Read the .xlsx file into a pandas DataFrame
    df = pd.read_excel(input_file)

    # Extract the sequence length from the first sequence (assumes all sequences have the same length)
    seq_length = len(df["Amino Acid Sequence"].iloc[0])

    # Ensure position_criteria matches the sequence length
    if len(position_criteria) != seq_length:
        raise ValueError(f"Position criteria length ({len(position_criteria)}) does not match sequence length ({seq_length}).")

    # Generate a regular expression based on position criteria
    regex_pattern = ''
    for i, criteria in enumerate(position_criteria):
        if criteria == 'x':
            regex_pattern += '.'  # Match any character
        else:
            regex_pattern += criteria  # Match the specified amino acid

    # Filter the DataFrame for sequences matching the pattern
    sorted_df = df[df["Amino Acid Sequence"].str.contains(regex_pattern)]
    unsorted_df = df[~df["Amino Acid Sequence"].str.contains(regex_pattern)]

    # Combine the sorted and unsorted DataFrames, adding a blank row in between
    combined_df = pd.concat([sorted_df, pd.DataFrame([['']*len(df.columns)], columns=df.columns), unsorted_df])

    # Write the combined data to the output file (temp)
    combined_df.to_excel(output_file, index=False)

    # Now, open the file again using openpyxl to apply highlighting
    wb = openpyxl.load_workbook(output_file)
    ws = wb.active

    # Yellow fill style for highlighting
    yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    # Iterate over the rows and check values in column 2 for highlighting
    for row in range(2, ws.max_row + 1):  # Start from row 2 to skip header
        count_in_file1 = ws.cell(row=row, column=2).value
        if count_in_file1 is not None and count_in_file1 < 100:
            for col in range(1, ws.max_column + 1):
                ws.cell(row=row, column=col).fill = yellow_fill

    # Save the workbook with the highlighting
    wb.save(output_file)
    print(f"Sorted data with highlighting saved to {output_file}")

# Specify input and output file paths
input_file = r'E105_E106.xlsx'
output_file = r'E105_E106_P21.xlsx'

# Set the position criteria (for example: 'K' for position 1, 'x' for position 2, 'G' for position 3 in a 3-residue sequence)
# For 4-residue sequences, add a 4th position if necessary
position_criteria = ['R', 'L', 'x']  # Modify this as needed

# Run the function to sort amino acid sequences and highlight rows
sort_amino_acid_sequences(input_file, output_file, position_criteria)


In [ ]:
#Takes all .xlxs in a given directory with reads before, reads after, fold-enrichment and adds Log2 and Log10 columns
#Demands 100 reads in the before pool; if >100 reads in before pool and 0 reads in after pool, fold-enrichment is calculated as if there was 1
#read in the output-i.e. approximate resolution limit of the sequencing.

import os
import math
import openpyxl

def fix_and_add_log_columns_to_excel(directory):
    # Traverse all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(directory, filename)
            print(f"Processing file: {filename}")

            # Load the workbook and select the active sheet
            workbook = openpyxl.load_workbook(file_path)
            sheet = workbook.active

            # Add headers for new columns if not already present
            sheet.cell(row=1, column=5, value="Log2(fold-enrichment)")
            sheet.cell(row=1, column=6, value="Log10(fold-enrichment)")

            # Iterate through the rows, assuming data starts from row 2 (row 1 is the header)
            for row in range(2, sheet.max_row + 1):
                try:
                    # Get the values in columns 2, 3, and 4
                    value_in_column_2 = sheet.cell(row=row, column=2).value
                    value_in_column_3 = sheet.cell(row=row, column=3).value
                    fold_enrichment = sheet.cell(row=row, column=4).value

                    if value_in_column_2 >= 100:
                        # If column 4 is 0 and column 3 is 0, update column 4
                        if fold_enrichment == 0 and value_in_column_3 == 0:
                            fold_enrichment = 1 / value_in_column_2
                            sheet.cell(row=row, column=4, value=fold_enrichment)

                        # Calculate log2 and log10 of column 4
                        if fold_enrichment > 0:
                            log2_value = math.log2(fold_enrichment)
                            log10_value = math.log10(fold_enrichment)
                        else:
                            log2_value = log10_value = None  # Handle cases where fold-enrichment is non-positive

                        # Write the values to columns 5 and 6
                        sheet.cell(row=row, column=5, value=log2_value)
                        sheet.cell(row=row, column=6, value=log10_value)
                    else:
                        # If column 2 is less than 100, leave columns 4, 5, and 6 blank
                        sheet.cell(row=row, column=4, value=None)
                        sheet.cell(row=row, column=5, value=None)
                        sheet.cell(row=row, column=6, value=None)

                except TypeError:
                    # If there is invalid data in column 2, 3, or 4, skip the row
                    print(f"Skipping row {row} due to invalid data.")

            # Save the modified workbook
            workbook.save(file_path)
            print(f"Updated file saved: {filename}")

if __name__ == "__main__":
    directory = r""
    fix_and_add_log_columns_to_excel(directory)
